# Turorial 1: Getting Started With TensorFlow

from: https://www.tensorflow.org/get_started/get_started

In [1]:
import tensorflow as tf

### constant nodes

In [2]:
# type of node: constant
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1, node2)

Tensor("Const:0", shape=(), dtype=float32) Tensor("Const_1:0", shape=(), dtype=float32)


In [3]:
# must run the graph to compute the nodes
sess = tf.Session()
print(sess.run([node1, node2]))

[3.0, 4.0]


In [4]:
# operations are also nodes (tf.add)
from __future__ import print_function
node3 = tf.add(node1, node2)
print("node3:", node3)
print("sess.run(node3):", sess.run(node3))

node3: Tensor("Add:0", shape=(), dtype=float32)
sess.run(node3): 7.0


### placeholder nodes (variables)

In [5]:
# place holder nodes can take input
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
adder_node = a + b  # + provides a shortcut for tf.add(a, b)
# assigning inputs and printing
print(sess.run(adder_node, {a: 3, b: 4.5}))
print(sess.run(adder_node, {a: [1, 3], b: [2, 4]}))
# making it more complext
add_and_triple = adder_node * 3.
print(sess.run(add_and_triple, {a: 3, b: 4.5}))   # notice 2 inputs only, at the "in nodes" of the graph

7.5
[ 3.  7.]
22.5


## Build a Model (=Build a Graph)

#### This is called the *construction* phase

1. Capture the model with equation containing trainable parameters <BR>
   Use Variable nodes for trainable parameters
2. Define loss function

In [6]:
# linear model: we will train W and b
W = tf.Variable([.3], dtype=tf.float32)        # initial value = .3
b = tf.Variable([-.3], dtype=tf.float32)       # initial values = -.3
x = tf.placeholder(tf.float32)
linear_model = W*x + b
# must initialize all variables
init = tf.global_variables_initializer()
sess.run(init)
# Since x is a placeholder, we can evaluate linear_model for several values of x simultaneously as follows
print(sess.run(linear_model, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


In [7]:
# a loss function will be defined in terms of variation from desired output y
y = tf.placeholder(tf.float32)
# we will need a loss function that we will want to minimize
squared_deltas = tf.square(linear_model - y)    # tensor with square error values
loss = tf.reduce_sum(squared_deltas)            # scalar with sum of square errors
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


## Train the model

#### This is called the *execution* phase

1. Choose optimizer
2. Train to minimize loss

In [8]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

In [9]:
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([W, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


# Complete Program

In [10]:
import tensorflow as tf

# Model parameters
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)

# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W*x + b
y = tf.placeholder(tf.float32)

# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares

# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# training data
x_train = [1, 2, 3, 4]
y_train = [0, -1, -2, -3]

# training loop
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init) # reset values to wrong
for i in range(1000):
  sess.run(train, {x: x_train, y: y_train})

# evaluate training accuracy
curr_W, curr_b, curr_loss = sess.run([W, b, loss], {x: x_train, y: y_train})
print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [-0.9999969] b: [ 0.99999082] loss: 5.69997e-11


# tf.estimator

In [15]:
# NumPy is often used to load, manipulate and preprocess data.
import numpy as np
import tensorflow as tf

# Declare list of features. We only have one numeric feature. There are many
# other types of columns that are more complicated and useful.
feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# An estimator is the front end to invoke training (fitting) and evaluation
# (inference). There are many predefined types like linear regression,
# linear classification, and many neural network classifiers and regressors.
# The following code provides an estimator that does linear regression.
estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# TensorFlow provides many helper methods to read and set up data sets.
# Here we use two data sets: one for training and one for evaluation
# We have to tell the function how many batches
# of data (num_epochs) we want and how big each batch should be.
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=None, shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, y_train, batch_size=4, num_epochs=1000, shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, y_eval, batch_size=4, num_epochs=1000, shuffle=False)

# We can invoke 1000 training steps by invoking the  method and passing the
# training data set.
estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.
train_metrics = estimator.evaluate(input_fn=train_input_fn)
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)
print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_ps_replicas': 0, '_is_chief': True, '_tf_random_seed': None, '_session_config': None, '_num_worker_replicas': 1, '_task_id': 0, '_keep_checkpoint_every_n_hours': 10000, '_task_type': 'worker', '_master': '', '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x1187ef208>, '_service': None, '_log_step_count_steps': 100, '_model_dir': '/var/folders/xr/3hcncdy943vfc7thtsyb6qfr0000gn/T/tmpexp3zgy7', '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_keep_checkpoint_max': 5, '_save_checkpoints_secs': 600}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/xr/3hcncdy943vfc7thtsyb6qfr0000gn/T/tmpexp3zgy7/model.ckpt.
INFO:tensorflow:loss = 18.0, step = 1
INFO:tensorflow:global_step/sec: 1262.91
INFO:tensorflow:loss = 0.365977, step = 101 (0.082 sec)
INFO:tensorflow:global_step/sec: 1394.13
INFO:tensorflow:loss = 0.0826078, step = 

In [47]:
# predictions 
x_pred = np.array([1., 2., 3., 4.])
pred_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_pred},  
    num_epochs=1, 
    shuffle=False)
predictions = list(estimator.predict(input_fn=pred_input_fn))
predictions

INFO:tensorflow:Restoring parameters from /var/folders/xr/3hcncdy943vfc7thtsyb6qfr0000gn/T/tmpexp3zgy7/model.ckpt-1000


[{'predictions': array([-0.00085551], dtype=float32)},
 {'predictions': array([-1.00038803], dtype=float32)},
 {'predictions': array([-1.99992061], dtype=float32)},
 {'predictions': array([-2.99945307], dtype=float32)}]